In [237]:
# Read table as dataframe, and then generate some features(numeric) to construct a 2D array.
import numpy as np
import pandas as pd
import csv
import py_stringmatching as sm


In [238]:
table = pd.read_csv('OUT.csv', quoting=csv.QUOTE_NONE)

In [239]:
table

,_id,ltable_id,rtable_id,Dark Age of Camelot: Shrouded Isles,ltable_release_day,ltable_release_year,ltable_esrb,ltable_genre,ltable_publisher,ltable_developer,...,rtable_title,rtable_release_day,rtable_release_year,rtable_esrb,rtable_genre,rtable_publisher,rtable_developer,rtable_META_rating,rtable_META_user_rating,label
0,5,1465,1,Grand Theft Auto: London 1969,May/June,1999,M,Action,Rockstar Games,DMA Design,...,Grand Theft Auto V,Apr 14,2015,M,Modern,Rockstar Games,Rockstar North,96,7.8,0
1,30,755,8,Command & Conquer 3 Tiberium Wars,March 28,2007,T,Strategy,Electronic Arts,EA Los Angeles,...,Command & Conquer,Aug 31,1995,T,Real-Time Strategy,Virgin Interactive,Westwood Studios,94,8.9,0
2,36,398,9,The Elder Scrolls III: Bloodmoon,June 3,2003,T,RPG,Bethesda Softworks,Bethesda Softworks,...,The Elder Scrolls V: Skyrim,Nov 11,2011,M,Role-Playing,Bethesda Softworks,Bethesda Game Studios,94,8.1,0
3,44,159,10,Mass Effect 3: Citadel,March 5,2013,M,RPG,Electronic Arts,BioWare,...,Mass Effect 2,Jan 26,2010,M,Role-Playing,Electronic Arts,BioWare,94,8.8,0
4,63,398,15,The Elder Scrolls III: Bloodmoon,June 3,2003,T,RPG,Bethesda Softworks,Bethesda Softworks,...,The Elder Scrolls IV: Oblivion,Mar 20,2006,M,Role-Playing,2K Games,Bethesda Softworks,94,8.1,0
5,65,70,16,Grim Fandango,Nov 2,1998,T,Adventure,LucasArts,LucasArts,...,Grim Fandango,Sep 30,1998,T,Adventure,LucasArts,LucasArts,94,9.1,1
6,69,517,19,Sid Meier's Civilization IV: Colonization,September 23,2008,E10+,Strategy,2K Games,Firaxis,...,Sid Meier's Civilization IV,Oct 25,2005,E10+,Strategy,2K Games,Firaxis Games,94,8.2,0
7,78,1389,19,Sid Meier's Civilization II: Test of Time,June,1999,E,Strategy,Hasbro Interactive,Microprose,...,Sid Meier's Civilization IV,Oct 25,2005,E10+,Strategy,2K Games,Firaxis Games,94,8.2,0
8,87,835,21,City of Heroes,April 27,2004,T,RPG,NCsoft,Cryptic Studios,...,Company of Heroes,Sep 13,2006,M,Strategy,THQ,Relic,93,8.8,0
9,101,3,26,Grand Theft Auto V,April 14,2015,M,Action,Rockstar Games,Rockstar North,...,Grand Theft Auto III,May 20,2002,M,Action Adventure,Rockstar Games,DMA Design Rockstar North,93,8.4,0


In [240]:
#Features: 
#    0: Whether the release years equal
#    1: Jaccard value of titles by word boundaries
#    2: Whether ESRBs equal 
#    3: Abs difference between IGN and META users (optional)
numFeatures = 3
numEntities = 394
fv = [[0 for x in range(numFeatures)] for y in range(numEntities)]

In [241]:
# Index 0: Whether the release years equal
for i in range(0,394):
    try:
        if int(table.iloc[i][5]) == table.iloc[i][14]:
            fv[i][0] = 1
    except Exception as inst:    
        print(i)
    

35
87
321


In [242]:
# Index 1: Jaccard value of titles by word boundaries
#for i in range(0,394):
#    try:
#        jac = sm.Jaccard()
#        sim_score = jac.get_sim_score(table.iloc[i][3].split(" "),table.iloc[i][12].split(" "))
#        fv[i][1] = sim_score
#    except Exception as inst:
#        print(i)

In [243]:
for i in range(0,394):
    try:
        qg3_tok = sm.QgramTokenizer(3)
        qgram1 = qg3_tok.tokenize(table.iloc[i][3])
        qgram2 = qg3_tok.tokenize(table.iloc[i][12])
        jac = sm.Jaccard()
        sim_score = jac.get_sim_score(qgram1,qgram2)
        fv[i][1] = sim_score
    except Exception as inst:
        print(i)
#sm.tokenizer.qgram_tokenizer.q_gramTokenizer(3) table.iloc[0][3]

14


In [244]:
#lev = sm.Levenshtein()
#for i in range(0,394):
#    try:
#        fv[i][3] = lev.get_sim_score(table.iloc[i][3],table.iloc[i][12])
#    except Exception as inst:
#        print(i)

In [245]:
# Index 2: Whether ESRBs equal 
for i in range(0,394):
    try:
        if table.iloc[i][6]==table.iloc[i][15]:
            fv[i][2] = 1
    except Exception as inst:
        print(i)

In [122]:
from sklearn import tree

clf = tree.DecisionTreeClassifier()
tg = table.as_matrix(columns=table.columns[21:])
sample = [0 for x in range(210)]
target = [0 for x in range(210)]


sample[0:210] = fv[:][0:210]
#sample[140:210] = fv[:][210:280]


target[0:210] = tg[0:210]
#target[140:210] = tg[210:280]


clf = clf.fit(sample,target)

prd = clf.predict(fv[:][210:280])

arr = [0 for x in range(70)]
for i in range(0,70):
    arr[i]=table.iloc[i+210][21]

# Calculate Precision
numPrd = 0
numGold = 0
for i in range(70):
    if(prd[i]==1):
        numPrd += 1
        if(arr[i]==1):
            numGold += 1 
print(numPrd)
print(numGold)

# Calculate Recall
numRcl = 0
numGold = 0
for i in range(70):
    if(arr[i]==1):
        numGold += 1
        if(prd[i]==1):
            numRcl += 1
print(numRcl)
print(numGold)

29
26
26
27


In [162]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
tg = table.as_matrix(columns=table.columns[21:])
sample = [0 for x in range(210)]
target = [0 for x in range(210)]


sample[0:210] = fv[:][0:210]
#sample[140:210] = fv[:][210:280]


target[0:210] = tg[0:210]
#target[140:210] = tg[210:280]


clf = clf.fit(sample,target)

prd = clf.predict(fv[:][210:280])

arr = [0 for x in range(70)]
for i in range(0,70):
    arr[i]=table.iloc[i+210][21]

# Calculate Precision
numPrd = 0
numGold = 0
for i in range(70):
    if(prd[i]==1):
        numPrd += 1
        if(arr[i]==1):
            numGold += 1 
print(numPrd)
print(numGold)

# Calculate Recall
numRcl = 0
numGold = 0
for i in range(70):
    if(arr[i]==1):
        numGold += 1
        if(prd[i]==1):
            numRcl += 1
print(numRcl)
print(numGold)

28
26
26
27


/u/z/h/zhenda/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:17: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


In [170]:
from sklearn import svm
clf = svm.SVC()
tg = table.as_matrix(columns=table.columns[21:])
sample = [0 for x in range(210)]
target = [0 for x in range(210)]


sample[0:140] = fv[:][0:140]
sample[140:210] = fv[:][210:280]


target[0:140] = tg[0:140]
target[140:210] = tg[210:280]


clf = clf.fit(sample,target)

prd = clf.predict(fv[:][140:210])

arr = [0 for x in range(70)]
for i in range(0,70):
    arr[i]=table.iloc[i+140][21]

# Calculate Precision
numPrd = 0
numGold = 0
for i in range(70):
    if(prd[i]==1):
        numPrd += 1
        if(arr[i]==1):
            numGold += 1 
print(numPrd)
print(numGold)

# Calculate Recall
numRcl = 0
numGold = 0
for i in range(70):
    if(arr[i]==1):
        numGold += 1
        if(prd[i]==1):
            numRcl += 1
print(numRcl)
print(numGold)

31
28
28
28


/u/z/h/zhenda/anaconda3/lib/python3.5/site-packages/sklearn/svm/base.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y_ = column_or_1d(y, warn=True)


In [175]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
tg = table.as_matrix(columns=table.columns[21:])
sample = [0 for x in range(210)]
target = [0 for x in range(210)]


sample[0:210] = fv[:][70:280]
#sample[70:210] = fv[:][140:280]


target[0:210] = tg[70:280]
#target[70:210] = tg[140:280]


clf = clf.fit(sample,target)

prd = clf.predict(fv[:][0:70])

arr = [0 for x in range(70)]
for i in range(0,70):
    arr[i]=table.iloc[i][21]

# Calculate Precision
numPrd = 0
numGold = 0
for i in range(70):
    if(prd[i]==1):
        numPrd += 1
        if(arr[i]==1):
            numGold += 1 
print(numPrd)
print(numGold)

# Calculate Recall
numRcl = 0
numGold = 0
for i in range(70):
    if(arr[i]==1):
        numGold += 1
        if(prd[i]==1):
            numRcl += 1
print(numRcl)
print(numGold)

18
17
17
17


/u/z/h/zhenda/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [180]:
from sklearn import linear_model
clf = linear_model.LogisticRegression()
tg = table.as_matrix(columns=table.columns[21:])
sample = [0 for x in range(210)]
target = [0 for x in range(210)]


sample[0:210] = fv[:][0:210]
#sample[140:210] = fv[:][210:280]


target[0:210] = tg[0:210]
#target[140:210] = tg[210:280]


clf = clf.fit(sample,target)

prd = clf.predict(fv[:][0:70])

arr = [0 for x in range(70)]
for i in range(0,70):
    arr[i]=table.iloc[i][21]

# Calculate Precision
numPrd = 0
numGold = 0
for i in range(70):
    if(prd[i]==1):
        numPrd += 1
        if(arr[i]==1):
            numGold += 1 
print(numPrd)
print(numGold)

# Calculate Recall
numRcl = 0
numGold = 0
for i in range(70):
    if(arr[i]==1):
        numGold += 1
        if(prd[i]==1):
            numRcl += 1
print(numRcl)
print(numGold)

18
17
17
17


/u/z/h/zhenda/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [250]:
#from sklearn import linear_model
#clf = linear_model.LogisticRegression()
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
tg = table.as_matrix(columns=table.columns[21:])
sample = [0 for x in range(140)]
target = [0 for x in range(140)]


sample[0:140] = fv[:][0:140]
#sample[140:210] = fv[:][210:280]


target[0:140] = tg[0:140]
#target[140:210] = tg[210:280]


clf = clf.fit(sample,target)

prd = clf.predict(fv[:][140:280])

arr = [0 for x in range(140)]
for i in range(0,140):
    arr[i]=table.iloc[i+140][21]

# Calculate Precision
numPrd = 0
numGold = 0
for i in range(140):
    if(prd[i]==1):
        numPrd += 1
        if(arr[i]==1):
            numGold += 1 
print(numPrd)
print(numGold)

# Calculate Recall
numRcl = 0
numGold = 0
for i in range(140):
    if(arr[i]==1):
        numGold += 1
        if(prd[i]==1):
            numRcl += 1
print(numRcl)
print(numGold)

58
53
53
55


/u/z/h/zhenda/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:19: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


In [223]:
for i in range(140):
    if(arr[i]!=prd[i]):
        print ("golden label is %d" %arr[i])
        print ('predict is %d' %prd[i])
        print(140+i)
        print(fv[:][140+i])

golden label is 1
predict is 0
145
[1, 0.4482758620689655, 1, 0.52]
golden label is 0
predict is 1
159
[0, 1.0, 1, 1.0]
golden label is 0
predict is 1
199
[1, 0.6351351351351351, 1, 0.6341463414634146]
golden label is 0
predict is 1
233
[1, 0.5352112676056338, 0, 0.7192982456140351]
golden label is 1
predict is 0
238
[1, 0.4864864864864865, 1, 0.5454545454545454]
golden label is 0
predict is 1
256
[1, 0.6521739130434783, 1, 0.7142857142857143]


In [268]:
from sklearn import linear_model
clf = linear_model.LogisticRegression()
tg = table.as_matrix(columns=table.columns[21:])
sample = [0 for x in range(280)]
target = [0 for x in range(280)]


sample[0:280] = fv[:][0:280]
#sample[140:210] = fv[:][210:280]


target[0:280] = tg[0:280]
#target[140:210] = tg[210:280]


clf = clf.fit(sample,target)

prd = clf.predict(fv[:][280:394])

arr = [0 for x in range(114)]
for i in range(0,114):
    arr[i]=table.iloc[i+280][21]

# Calculate Precision
numPrd = 0
numGold = 0
for i in range(114):
    if(prd[i]==1):
        numPrd += 1
        if(arr[i]==1):
            numGold += 1 
print(numPrd)
print(numGold)

# Calculate Recall
numRcl = 0
numGold = 0
for i in range(114):
    if(arr[i]==1):
        numGold += 1
        if(prd[i]==1):
            numRcl += 1
print(numRcl)
print(numGold)

61
53
53
54


/u/z/h/zhenda/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
